# UHF

## Imports

In [1]:
import numpy as np
import scipy.linalg as spla
import pyscf
from pyscf import gto, scf
import matplotlib.pyplot as plt
import time
%matplotlib notebook

## Some useful resources:
 - Szabo and Ostlund Chapter 3 (for algorithm see page 146)
 - [Notes by David Sherrill](http://vergil.chemistry.gatech.edu/notes/hf-intro/hf-intro.html)
 - [Psi4Numpy SCF page](https://github.com/psi4/psi4numpy/tree/master/Tutorials/03_Hartree-Fock)

## The UHF algorithm from Szabo and Ostlund (NEEDS UPDATED):
 1. Specify a molecule (coordinates $\{R_A\}$, atomic numbers $\{Z_A\}$, number electrons $N$) and atomic orbital basis $\{\phi_\mu\}$.
 2. Calculate molecular integrals over AOs ( overlap $S_{\mu\nu}$, core Hamiltonian $H^{\mathrm{core}}_{\mu\nu}$, and 2  electron integrals $(\mu \nu | \lambda \sigma)$ ).
 3. Diagonalize the overlap matrix $S$ to obtain the transformation matrix $X$.
 4. Make a guess at the original density matrix $P$.
 5. Calculate the intermediate matrix $G$ using the density matrix $P$ and the two electron integrals $(\mu \nu | \lambda \sigma)$.
 6. Construct the Fock matrix $F$ from the core hamiltonian $H^{\mathrm{core}}_{\mu\nu}$ and the intermediate matrix $G$.
 7. Transform the Fock matrix $F' = X^\dagger F X$.
 8. Diagonalize the Fock matrix to get orbital energies $\epsilon$ and molecular orbitals (in the transformed basis) $C'$.
 9. Transform the molecular orbitals back to the AO basis $C = X C'$.
 10. Form a new guess at the density matrix $P$ using $C$.
 11. Check for convergence. (Are the changes in energy and/or density smaller than some threshold?) If not, return to step 5.
 12. If converged, use the molecular orbitals $C$, density matrix $P$, and Fock matrix $F$ to calculate observables like the total Energy, etc.

## Quick note
The reason we need to calculate the transformation matrix $X$ is because the atomic orbital basis is not orthonormal by default. This means without transformation we would need to solve a generalized eigenvalue problem $FC = ESC$. If we use scipy to solve this generalized eigenvalue problem we can simply the SCF algorithm.
## Simplified SCF
 1. Specify a molecule (coordinates $\{R_A\}$, atomic numbers $\{Z_A\}$, number electrons $N$) and atomic orbital basis $\{\phi_\mu\}$.
 2. Calculate molecular integrals over AOs ( overlap $S_{\mu\nu}$, core Hamiltonian $H^{\mathrm{core}}_{\mu\nu}$, and 2  electron integrals $(\mu \nu | \lambda \sigma)$ ).
 3. Make a guess at the original density matrix $P$.
 4. Calculate the intermediate matrix $G$ using the density matrix $P$ and the two electron integrals $(\mu \nu | \lambda \sigma)$.
 5. Construct the Fock matrix $F$ from the core hamiltonian $H^{\mathrm{core}}_{\mu\nu}$ and the intermediate matrix $G$. 
 6. Solve the generalized eigenvalue problem using the Fock matrix $F$ and the overlap matrix $S$ to get orbital energies $\epsilon$ and molecular orbitals.
 7. Form a new guess at the density matrix $P$ using $C$.
 8. Check for convergence. (Are the changes in energy and/or density smaller than some threshold?) If not, return to step 4.
 9. If converged, use the molecular orbitals $C$, density matrix $P$, and Fock matrix $F$ to calculate observables like the total Energy, etc.


# STEP 1 : Specify the molecule

In [2]:
# start timer
start_time = time.time()
# define molecule
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit = "Ang",
    verbose=0,
    symmetry=False,
    spin = 1,
    charge = -1
)
# get number of atomic orbitals
num_ao = mol.nao_nr()
# get number of electrons
num_elec_alpha, num_elec_beta = mol.nelec
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc = mol.energy_nuc()

# STEP 2 : Calculate molecular integrals 

Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H = T + V $$

Two electron integrals

$$ (\mu\nu|\lambda\sigma) = \int dr_1 dr_2 \phi^*_{\mu}(r_1) \phi_{\nu}(r_1) r_{12}^{-1} \phi_{\lambda}(r_2) \phi_{\sigma}(r_2) $$


In [3]:
# calculate overlap integrals
S = mol.intor('cint1e_ovlp_sph')
# calculate kinetic energy integrals
T = mol.intor('cint1e_kin_sph')
# calculate nuclear attraction integrals
V = mol.intor('cint1e_nuc_sph')
# form core Hamiltonian
H = T + V
# calculate two electron integrals
eri = mol.intor('cint2e_sph',aosym='s8')
# since we are using the 8 fold symmetry of the 2 electron integrals
# the functions below will help us when accessing elements
__idx2_cache = {}
def idx2(i, j):
    if (i, j) in __idx2_cache:
        return __idx2_cache[i, j]
    elif i >= j:
        __idx2_cache[i, j] = int(i*(i+1)/2+j)
    else:
        __idx2_cache[i, j] = int(j*(j+1)/2+i)
    return __idx2_cache[i, j]
def idx4(i, j, k, l):
    return idx2(idx2(i, j), idx2(k, l))

# STEP 3 : Form guess density matrix

In [4]:
# set inital density matrix to zero
D_total = np.zeros((num_ao,num_ao))
D_alpha = np.zeros((num_ao,num_ao))
D_beta = np.zeros((num_ao,num_ao))

# STEPS 4 - 8 : SCF loop

 4. Calculate the intermediate matrix $G$ using the density matrix $P$ and the two electron integrals $(\mu \nu | \lambda \sigma)$.
 
 $$G_{\mu\nu} = \sum_{\lambda\sigma}^{\mathrm{num\_ao}} P_{\lambda \sigma}[2(\mu\nu|\lambda\sigma)-(\mu\lambda|\nu\sigma)]$$ 
 
 5. Construct the Fock matrix $F$ from the core hamiltonian $H^{\mathrm{core}}_{\mu\nu}$ and the intermediate matrix $G$. 
 
 $$ F = H + G $$
 
 6. Solve the generalized eigenvalue problem using the Fock matrix $F$ and the overlap matrix $S$ to get orbital energies $\epsilon$ and molecular orbitals.
 
 $$F C  = E  S C $$
 
 7. Form a new guess at the density matrix $P$ using $C$.
 
 $$ P_{\mu\nu} = \sum_{i}^{\mathrm{num\_elec}/2} C_{\mu i} C_{\nu i} $$
 
 8. Check for convergence. (Are the changes in energy and/or density smaller than some threshold?) If not, return to step 4.
 
 $$ E_{\mathrm{elec}}  = \sum^{\mathrm{num\_ao}}_{\mu\nu} P_{\mu\nu} (H_{\mu\nu} + F_{\mu\nu})  $$
 $$ \Delta E = E_{\mathrm{new}} - E_{\mathrm{old}} $$
 $$ |\Delta P| = \left[ \sum^{\mathrm{num\_ao}}_{\mu\nu} [P^{\mathrm{new}}_{\mu\nu} - P_{\mu\nu}^{\mathrm{old}}]^2 \right]^{1/2}$$
 
 9. If converged, use the molecular orbitals $C$, density matrix $P$, and Fock matrix $F$ to calculate observables like the total Energy, etc.
 
 $$ E_{\mathrm{total}} = V_{\mathrm{NN}} + E_{\mathrm{elec}} $$

In [5]:
# 2 helper functions for printing during SCF
def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("Iter", "Time(s)", "RMSC DM", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("****", "*******", "*******", "*******", "******")))
def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num, iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_elec)))

# set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# loop variables
iteration_num = 0
E_total = 0
E_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False


In [6]:
print_start_iterations()
while (not converged and not exceeded_iterations):
    # store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_elec
    D_total_last = np.copy(D_total)
    # form G matrix
    G_alpha = np.zeros((num_ao,num_ao))
    G_beta = np.zeros((num_ao,num_ao))
    
    #########################################################
    # FILL IN HOW TO MAKE THE G MATRICES HERE
    #########################################################
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G_alpha[i,j] += D_alpha[k,l] * ((2.0*(eri[idx4(i,j,k,l)])) - (eri[idx4(i,k,j,l)]))
                    G_beta[i,j] += D_beta[k,l] * ((2.0*(eri[idx4(i,j,k,l)])) - (eri[idx4(i,k,j,l)]))
    
    # build fock matrices
    
    F_alpha  = H + G_alpha
    F_beta  = H + G_beta
    
    #########################################################
    # FILL IN HOW TO MAKE THE FOCK MATRICES HERE
    #########################################################
    
    # solve the generalized eigenvalue problem
    E_orbitals_alpha, C_alpha = spla.eigh(F_alpha,S)
    E_orbitals_beta, C_beta = spla.eigh(F_beta,S)
    # compute new density matrix
    D_alpha = np.zeros((num_ao,num_ao))
    D_beta = np.zeros((num_ao,num_ao))
    
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D_alpha[i,j] +=  C_alpha[i,k] * C_alpha[j,k]
            for k in range(num_elec_beta):
                D_beta[i,j] +=  C_beta[i,k] * C_beta[j,k]
    D_total = D_alpha + D_beta
    #########################################################
    # FILL IN HOW TO MAKE THE DENSITY MATRICES HERE
    #########################################################
    E_elec = 0.5 * np.sum(np.multiply((D_total), H) + np.multiply(D_alpha, F_alpha) + np.multiply(D_beta, F_beta))
    # calculate electronic energy
    
    #########################################################
    # FILL IN HOW TO CALCULATE THE ELECTRONIC ENERGY HERE
    #########################################################
    
    # calculate energy change of iteration
    iteration_E_diff = np.abs(E_elec - E_elec_last)
    # rms change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D_total - D_total_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM): 
        converged = True
    if(iteration_num == iteration_max):
        exceeded_iterations = True

           Iter      Time(s)      RMSC DM      delta E       E_elec            
           ****      *******      *******      *******       ******            
              1     0.047132  5.91371E+00  1.31600E+02  -131.599702            
              2     0.018087  2.95622E+00  5.14128E+01   -80.186891            
              3     0.018034  1.16308E-01  2.98211E+00   -83.169005            
              4     0.019355  2.50466E-02  5.78616E-02   -83.111144            
              5     0.017451  4.77989E-03  3.16961E-02   -83.142840            
              6     0.015788  2.30330E-03  2.38236E-03   -83.145222            
              7     0.019115  1.10232E-03  9.42227E-04   -83.146164            
              8     0.014816  5.27878E-04  2.44228E-04   -83.146409            
              9     0.010818  2.51209E-04  7.92941E-05   -83.146488            
             10     0.010585  1.18480E-04  2.53767E-05   -83.146513            
             11     0.010740  5.52795E-0

# STEP 9 : Calculate Observables

In [7]:
# calculate total energy
E_total = E_elec + E_nuc
####################################################
# FILL IN HOW TO CALCULATE THE TOTAL ENERGY HERE
####################################################

In [8]:
print("{:^79}".format("Total Energy : {:>11f}".format(E_total)))

                          Total Energy :  -73.951661                           
